In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import os

In [ ]:
words = None
if False:
    from nltk.corpus import gutenberg as corpus
    words = corpus.words()
    vocabulary = vocabulary[803:-2]
    # first 803 and last 2 words are punctuation signs, numbers and underscored 
    # words like _home_
else:
    # Using the whole corpus means too much work to later compute the words at a 
    # distance for each word in the vocabulary, so we load just one book
    import nltk
    nltk.download('gutenberg')
    words = nltk.corpus.gutenberg.words('austen-emma.txt')
    # using all the words in the book will take 25-30 min. to process later
    # so we limit its number for the moment.
    words = words[:10000] # cell [15] will take 1.5 min. 
    #words = words[:100000] # cell [15] will take 15 min. 


In [ ]:
vocabulary = list(set(words))
vocabulary.sort()
# get rid of some non-words like ',' '--' '['
idx_first_word = vocabulary.index('A')
vocabulary = vocabulary[idx_first_word:]
# plus some more annoying non-words
vocabulary.remove('[')
vocabulary.remove(']')
vocabulary.remove('`')
vocabulary.remove('II')
vocabulary = np.array(vocabulary)
print(vocabulary)
print('Vocabulary size {} words'.format(len(vocabulary)))

For each word in the vocabulary find the nearest words = at Levenshtein distance up to ``MAX_DIST``. This is a long computation, $O(n^2)$ for $n$ size of the vocabulary. We try to speed up it a little : if $\text{dist}(w_1, w_2) \leq d$ then $|\text{len}(w_1) - \text{len}(w_2)| \leq d$. This reduces the candidate words in the vocabulary for which to compute the distance to each word.
We save the resulting dictionary to avoid recomputing it each time.

In [ ]:
from nltk.metrics.distance import edit_distance

def levenshtein(s1, s2):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

word_lengths = np.array([len(w) for w in vocabulary])
dict_lengths = {}
for l in range(min(word_lengths), max(word_lengths)+1):
    dict_lengths[l] = vocabulary[word_lengths==l] # needs vocabulary to be a numpy array

min_length = min(dict_lengths.keys())
max_length = max(dict_lengths.keys())

MAX_DIST = 1
fname = 'close_words_{}.pkl'.format(MAX_DIST)
if not os.path.exists(fname):
    close_words = {}
    for word in tqdm(vocabulary):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - MAX_DIST)
        d2 = min(max_length, length + MAX_DIST)
        for d in range(d1, d2+1):
            candidate_words.extend(dict_lengths[d])
        close_words[word] = [w for w in candidate_words if levenshtein(word,w) <= MAX_DIST]

    with open(fname,'wb') as f:
        pickle.dump(close_words, f)
else:
    close_words = pickle.load(open(fname,'rb'))

Given one sentence $X$, which is a list of words, build the candidates to correct sentence $C(X)$ assuming at most one word is mispelled.

In [ ]:
#sentence = 'Only two of the apples'
sentence = 'I wish you where here'
X = sentence.split(' ')
for x in X:
  assert x in vocabulary, 'All the words in the sentence must belong to the '\
      + 'vocabulary, {} doesn\'t'.format(x)
CX = [X] # no errors
for i in range(len(X)): # one mispelled word at a time
    if X[i] in vocabulary: 
        for cw in close_words[X[i]]:
            if cw != X[i]:
                C = X.copy()
                C[i] = cw
                CX.append(C)
    else:
        pass # let it be as is
for W in CX:
    print('\t'.join(W))

Likelihood $P(X | W) = \prod_{i=1}^n p(x_i | w_i)$ where $n$ is number of words in $X$ (same as in $W$), and $p(x | w)$  is Eq. B.8. $X$ is the written sentence, $W$ are the candidate sentences in $C(X)$. Each $W$ contains zero (ie, $W=X$) or at most one mispelled word, and in this case the mispelled word $w_i$ is at a Levenshtein distance $1...$ ``MAX_DIST`` of the written word $x_i$ 

In [ ]:
alpha = 0.95
likelihoods = []
for W in CX:
    PXW = 1.0
    #print(X,W)
    for x,w in zip(X,W):
        if w==x:
            pxw = alpha
        else:
            close_to_x = close_words[x] # includes x itself
            pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
        PXW *= pxw
    likelihoods.append(PXW)

likelihoods = np.array(likelihoods)
idx_most_likely = likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X')
print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
print('Likelihoods')
num_candidates = len(CX)
for i in range(num_candidates):
    print('\t'.join(CX[i]), '\t', likelihoods[i])


Priors $P(W)$ for all $W \in C(X)$ computed by a LM model, for instance tri-grams (on this same corpus). We've chosen the stupid backoff version.

In [ ]:
nltk.download('punkt')

sents = nltk.corpus.gutenberg.sents('austen-emma.txt')
text = []
for s in tqdm(sents):
    text.append(s[:-1]) # except ending point

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import StupidBackoff

n=3
lm = StupidBackoff(alpha=0.4, order=n)
train, vocab = padded_everygram_pipeline(n, text)
lm.fit(train, vocab)
print(lm.counts)

In [ ]:
def P(W, verbose=False):
    S = ['<s>', '<s>',] + W + ['</s>'] 
    if verbose: print(S)
    num_words = len(S)
    PW = 1.0
    for i in range(2,num_words-1): # omit </s> because likelihoods don't have it
        score = lm.score(S[i], [S[i-2], S[i-1]])
        if verbose: print('P({} | {}, {}) = {}'.format(S[i], S[i-2], S[i-1], score))
        PW *= score
    return PW

print('X =', X)
score = P(X, verbose=True)
print('P(X) = {}'.format(score))
print('\nW = CX[1] = {}'.format(CX[1]))
score = P(CX[1], verbose=True)
print('P(W) = {}'.format(score))

In [ ]:
priors = []
for W in CX:
    priors.append(P(W))
    
posteriors = np.array(priors)*np.array(likelihoods)
idx_best_post = np.argmax(posteriors)

for i in range(num_candidates):
    best = '<----' if i==idx_best_post else ''
    print('\t'.join(CX[i]), '\tlikelihood={}\tprior={}\tposterior={} {}'
          .format(likelihoods[i], priors[i], posteriors[i],best))

print('\nThe original sentence was')
print('\t' + ' '.join(X))
print('The right sentence is')
print('\t' + ' '.join(CX[idx_best_post]))

That's it! Now, let's try this code and do some modifications:
1. change probabilities by log of probabilities to get scores more easy to interpret
1. relax the assumption that in a sentence there's at most 1 mispelled word, what if we suppose there may be at most 2 ?
1. and what is the effect of $\alpha$ ? change it to 0.9, 0.8...
1. make a long (>20) list of sentences each with 1 or 2 mispelled words to assess the speller : does it really work ? 
1. same for a second list of sentences withouth any spelling error (but different from any one in the selected part of the book), are there any errors ?
1. extra points: what if you change the tri-gram language model by the neural network of exercise 2 ?